In [2]:
model = "dllModel"

In [3]:
import os

In [4]:
dll = os.path.join(build_root, model+"_win64.dll")
header = os.path.join(build_root, model+"_ert_shrlib_rtw", model+".h")

NameError: name 'build_root' is not defined

In [5]:
assert os.path.exists(dll)
assert os.path.exists(header)

NameError: name 'dll' is not defined

In [6]:
cast_xml_root = os.path.abspath(os.path.join(os.path.curdir, 'castxml'))

In [7]:
import pygccxml

from pygccxml import utils
from pygccxml import declarations
from pygccxml import parser
import ctypes
import os

In [8]:
# Configure the xml generator
xml_generator_config = parser.xml_generator_configuration_t(
    xml_generator_path=r"castxml\bin\castxml.exe",
    xml_generator="castxml",
    compiler_path=r"C:\MATLAB\SupportPackages\R2015b\MW_MinGW_4_9\bin\gcc.exe")

In [9]:
decls = parser.parse([header], xml_generator_config)
ns = decls[0]

NameError: name 'header' is not defined

In [10]:
def ctypes_cppname(CPPNAME):
    if CPPNAME.endswith('_T'):
        return CPPNAME
    type_map = {'long long int': 'c_longlong',
                'long long unsigned int': 'c_ulonglong',
                'short unsigned int': 'c_ushort',
                'int': 'c_int',
                'unsigned int': 'c_uint',
                'long int': 'c_long',
                'long unsigned int': 'c_ulong',
                'char': 'c_char',
                'char': 'c_char',
                'signed char': 'c_byte',
                'unsigned char': 'c_ubyte',
                'float': 'c_float',
                'double': 'c_double',
                'short int': 'c_short'}
    
    for cppname, ctype in type_map.items():
        if cppname == CPPNAME:
            return "ctypes."+ctype
    
    raise(Exception("Unhandled data type: "+CPPNAME))

print("# Standard Types")
for typedef in ns.typedefs():
    if hasattr(typedef.decl_type, 'CPPNAME'):
        print("{} = {}".format(typedef.name, ctypes_cppname(typedef.decl_type.CPPNAME)))

# Standard Types


NameError: name 'ns' is not defined

In [11]:
# Standard Types
size_t = ctypes.c_ulonglong
ssize_t = ctypes.c_longlong
intptr_t = ctypes.c_longlong
uintptr_t = ctypes.c_ulonglong
ptrdiff_t = ctypes.c_longlong
wint_t = ctypes.c_ushort
wctype_t = ctypes.c_ushort
errno_t = ctypes.c_int
__time32_t = ctypes.c_long
__time64_t = ctypes.c_longlong
int8_T = ctypes.c_byte
uint8_T = ctypes.c_ubyte
int16_T = ctypes.c_short
uint16_T = ctypes.c_ushort
int32_T = ctypes.c_int
uint32_T = ctypes.c_uint
int64_T = ctypes.c_longlong
uint64_T = ctypes.c_ulonglong
real32_T = ctypes.c_float
real64_T = ctypes.c_double
real_T = ctypes.c_double
time_T = ctypes.c_double
boolean_T = ctypes.c_ubyte
int_T = ctypes.c_int
uint_T = ctypes.c_uint
ulong_T = ctypes.c_ulong
ulonglong_T = ctypes.c_ulonglong
char_T = ctypes.c_char
uchar_T = ctypes.c_ubyte

In [12]:
def var_type(decl_type):
    if hasattr(decl_type, 'CPPNAME'):
        return(ctypes_cppname(decl_type.CPPNAME))
    if pygccxml.declarations.is_array(decl_type):
        if hasattr(decl_type.base, "CPPNAME"):
            ctype = ctypes_cppname(decl_type.base.CPPNAME)
        elif isinstance(decl_type.base, pygccxml.declarations.cpptypes.declarated_t):
            ctype = decl_type.decl_string.split(":")[-1]
        return "{}*{}".format(ctype.base, decl_type.size)
    if pygccxml.declarations.is_pointer(decl_type):
        return "ctypes.POINTER({})".format(var_type(declarations.remove_pointer(decl_type)))
    if pygccxml.declarations.is_volatile(decl_type):
        return "{}".format(var_type(declarations.remove_volatile(decl_type)))
    if pygccxml.declarations.is_const(decl_type):
        return "{}".format(var_type(declarations.remove_const(decl_type)))
    if isinstance(decl_type, pygccxml.declarations.cpptypes.declarated_t):
        return decl_type.decl_string.split(":")[-1]
    raise Exception(decl_type)
    
def struct_tdef(struct, ns):
    for tdef in ns.typedefs():
        try:
            if tdef.decl_type.declaration == struct:
                return tdef.name
        except:
            continue
    return None

In [13]:
class tag_RTM_dllModel2_T(ctypes.Structure):
    class Timing(ctypes.Structure):
        __fields_ = [
            ('tockTick0', uint32_T),
            ("tockTickH0", uint32_T)
        ]
    _fields_ = [
        ("errorStatus", ctypes.POINTER(ctypes.c_char)),
        ("Timing", Timing)
    ]

In [14]:
externs = ns.variables(lambda v: v.type_qualifiers.has_extern)
for extern in externs:
#    extern_type = extern.decl_type.declaration.name
    print("{} = {}.in_dll(dll, '{}')".format(extern.name, var_type(extern.decl_type), extern.name))

NameError: name 'ns' is not defined

In [15]:
class Timing(ctypes.Structure):
    _fields_ = [     
        ("clockTick0", uint32_T),
        ("clockTickH0", uint32_T),   
    ]

class tag_RTM_dllModel2_T(ctypes.Structure):
    _fields_ = [
        ("errorStatus", ctypes.POINTER(ctypes.c_char)),
        ("Timing", Timing),
    ]

In [16]:
def vardump(var):
    fields = dir(var)
    for field in fields:
        if field.startswith("_"):
            continue
        try:
            f = getattr(var, field)
        except:
            continue
        try:
            print("{}(): {}".format(field, f()))
        except:
            try:
                print("{}: {}<{}>".format(field, type(f), f))
            except:
                pass